<a href="https://colab.research.google.com/github/OneFineStarstuff/State-of-the-Art/blob/main/Generative_Adversarial_Networks_(GANs).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim

class Generator(nn.Module):
    def __init__(self, latent_dim, output_dim):
        super(Generator, self).__init__()
        self.net = nn.Sequential(
            nn.Linear(latent_dim, 256),
            nn.ReLU(),
            nn.Linear(256, 512),
            nn.ReLU(),
            nn.Linear(512, output_dim),
            nn.Tanh()
        )

    def forward(self, z):
        return self.net(z)

class Discriminator(nn.Module):
    def __init__(self, input_dim):
        super(Discriminator, self).__init__()
        self.net = nn.Sequential(
            nn.Linear(input_dim, 512),
            nn.LeakyReLU(0.2),
            nn.Linear(512, 256),
            nn.LeakyReLU(0.2),
            nn.Linear(256, 1),
            nn.Sigmoid()
        )

    def forward(self, x):
        return self.net(x)

latent_dim = 100
output_dim = 28 * 28  # MNIST image size
generator = Generator(latent_dim, output_dim)  # Removed .to('cuda')
discriminator = Discriminator(output_dim)  # Removed .to('cuda')

optimizer_G = optim.Adam(generator.parameters(), lr=0.0002)
optimizer_D = optim.Adam(discriminator.parameters(), lr=0.0002)
criterion = nn.BCELoss()

for epoch in range(20000):
    # Training Discriminator
    z = torch.randn(32, latent_dim)
    fake_images = generator(z)
    real_images = torch.randn(32, output_dim)  # Dummy real data
    real_labels = torch.ones(32, 1)
    fake_labels = torch.zeros(32, 1)

    optimizer_D.zero_grad()
    real_loss = criterion(discriminator(real_images), real_labels)
    fake_loss = criterion(discriminator(fake_images.detach()), fake_labels)
    d_loss = real_loss + fake_loss
    d_loss.backward()
    optimizer_D.step()

    # Training Generator
    optimizer_G.zero_grad()
    g_loss = criterion(discriminator(fake_images), real_labels)
    g_loss.backward()
    optimizer_G.step()

    if epoch % 1000 == 0:
        print(f"Epoch {epoch}, D Loss: {d_loss.item()}, G Loss: {g_loss.item()}")